# Zusatzkapitel - Wie baue ich mein eigenes Wikipediakorpus?

## Z.1. Kapitelübersicht <a class="anchor" id="Z-1"/>

In diesem Kapitel zeige ich, wie man mithilfe meiner selbsterstellen CLI-Tools `wikiparser.py` und `data_postprocess.py` ein solides deutsches Wikipedia-Korpus bauen kann. Dieses Korpus ist dabei genauso wie das Korpus, welches in dieser Tutorialreihe genutzt wurde, aufgebaut. Die Klassen sind Wikipedia-Kategorien und die Texte Wikipedia-Artikel.


<b>Abschnittsübersicht</b>:<br>
[Z.1. Kapitelübersicht](#Z-1)<br>
[Z.2. Erstellung der JSON-Datei mit Wikipediakategorien](#Z-2)<br>
[Z.3. Nutzung von wikiparser.py](#Z-3)<br>
[Z.4. Nutzung von data_postprocess.py](#Z-4)<br>

## Z.2. Erstellung der JSON-Datei mit Wikipediakategorien <a class="anchor" id="Z-2"/>

Schauen wir uns zunächst die JSON-Datei `testwikicatogeries.json` im Order `data` an. Sie hat folgenden Inhalt:<br>

```
{
 "Fernsehserie nach Staat": ["Kategorie:Fernsehserie (Australien)", "Kategorie:Fernsehserie (Belgien)"], 
 "Oper": ["Kategorie:Opernarie", "Kategorie:Bayreuther_Festspiele"]
}
``` 

Sie besteht aus einem Dictionary, bei dem die keys Wikipedia-Kategorien und die values Listen von untergeordneten Wikipedia-Kategorien sind. Um ein eigenes Korpus mit der gleichen Form zu bauen, muss eine solche JSON-Datei erstellt werden. Sind keine Unterkategorien erwünscht, muss als einziges Listenelement die Kategorie angegeben werden. Wichtig ist, dass vor den Kategorien in der values-Liste das Präfix "Kategorie:" steht.<br>

Doch wo können wir sehen, welche Kategorien es bei Wikipedia gibt? Die Sortierung von Artikeln nach Kategorien ist bei Wikipedia etwas versteckt. Sie findet sich unter der Spezial-Seite <a href="https://de.wikipedia.org/wiki/Spezial:Kategorien">Kategorien</a>. Dort kann man mithilfe der Suchmaske Kategorien finden. In Klammern steht immer die Anzahl der verfügbaren Artikel und/oder der Unterkategorien. Sollte ein nicht zu kleines Korpus erstellt werden, sollten Kategorien mit einer größeren Anzahl an Artikeln und/oder Unterkategorien ausgewählt werden.<br>

Geben wir einmal den Suchbegriff "Informatik" ein. Es existieren 10 Unterkategorien zu der Kategorie und 86 weitere Artikel (siehe <a href="https://de.wikipedia.org/wiki/Kategorie:Informatik">Kategorie:Informatik</a>). Manche Unterkategorien bestehen dabei aus nur wenigen Artikeln, diese lohnen sich meistens nicht für das Korpus. Ich würde in diesem Fall wahrscheinlich die etwas speziellere Kategorie <a href="https://de.wikipedia.org/wiki/Kategorie:Informatik_nach_Fachgebiet">"Informatik nach Fachgebiet"</a> wählen und ihre Unterkategorien in die values-Liste schreiben.<br>

Die Erstellung der JSON-Datei erfordert am meisten Arbeit und kann natürlich nicht nur händisch, sondern auch mit selbst geschriebenen Programmen und Parsern erstellt werden. Ich empfehle, einen Blick auf das Modul <a href="https://pypi.org/project/Wikipedia-API/">Wikipedia-API</a> zu werfen. Ich habe es genutzt, um die Wikipedia-Artikel im meinem CLI-Tool `wikiparser.py` zu parsen und zu extrahieren.

## Z.3. Nutzung von `wikiparser.py` <a class="anchor" id="Z-3"/>

Das CLI-Tool `wikiparser.py` kann wie alle anderen CLI-Tools über das Terminal geladen werden.

<img src="tutorialdata/img/wikiparser_1.png" alt="Wikiparser CLI-Tool Screenshot" align="center" width="600px;"/>

Es hat ein verpflichtendes Argument `path`. Dort muss der Pfad zu JSON-Datei angegeben werden. Optionale Argumente sind `--max_articles` und `--save_date`. `--max_articles` gibt die maximale Anzahl an Artikeln an, die geparst werden, um das Parsing zu beschleunigen und das Korpus nicht zu groß werden zu lassen. Der Standardwert ist 240. Dies sollte auch angegeben werden, wenn kein unausgeglichener Datensatz erwünscht ist (siehe Kapitel 5). Als Wert sollte dabei für einen ausgeglichenen Datensatz die niedrigste Anzahl an Artikeln einer Kategorie genommen werden. Ich würde persönlich empfehlen, den Wert etwas höher zu setzen als Artikel pro Kategorie erwünscht sind, da Artikel, die kleiner als 100 Tokens sind, vom Tool nicht geparst werden. Artikel, die mehr als 2000 Tokens haben, werden zugeschnitten. Fügt man dem CLI-Tool das Argument `--save_date` hinzu, wird das aktuelle Datum in die erstellte csv-Datei geschrieben.<br>

Funktionert alles, wird ein Fortschrittsbalken für jede Kategorie angezeigt. Die erstellte csv-Datei wird im Ordner `data/corpora` gespeichert.

<img src="tutorialdata/img/wikiparser_2.png" alt="Wikiparser CLI-Tool Screenshot 2" align="center" width="600px;"/>

<b>Besonderheiten vom `wikiparser.py`</b>:
- Folgende Sektionen von Wikipedia-Artikeln werden nicht gespeichert: 
    - "Literatur"
    - "Weblinks"
    - "Einzelnachweis" / "Einzelnachweise"
    - "Siehe auch"
- Artikel mit weniger als 100 Tokens werden beim Parsen übersprungen
- Artikel mit mehr als 2000 Tokens werden beim Parsen auf 2000 Tokens gekürzt
- Je nach Anzahl der Kategorien erhält die ausgegebene csv-Datei folgende Namen:
    - smallwikicorpus → 10 oder weniger Kategorien
    - wikicorpus → mehr als 10 und weniger als 51 Kategorien
    - bigwikicorpus → mehr als 50 Kategorien
- In `parserutils.py` befindet sich die auskommentierte Zeilen `# article_dict["summary"] = article.summary`. Wird die Kommentierung entfernt, kann auch die Zusammenfassung zu Beginn jedes Wikipedia-Artikel in der csv-Datei als einzelne Spalte gespeichert werden.

## Z.4. Nutzung von `data_postprocess.py` <a class="anchor" id="Z-4"/>

Auch das CLI-Tool `data_postprocess.py` kann wie `wikiparser.py` über das Terminal geladen werden. Es hat ebenfalls das verpflichtende Argument `path`, welches diesmal jedoch auf das Korpus verweisen soll. `data_postprocess.py` hat einige Argumente, die hier im Detail erläutert werden sollen:<br>

`--drop_duplicates`: Wird dieses Argument angegeben, werden alle doppelten Artikel entfernt. Da Wikipedia-Artikel mehreren Kategorien angehören können, kann es relativ schnell zu Dopplungen bei den Artikeln kommen. Aufgrund dieser Möglichkeit würde ich empfehlen, `--max_articles` von `wikiparser.py` höher zu setzen als die tatsächliche gewünschte Anzahl an Artikeln.<br>

`--unify_articles_amount`: Wird dieses Argument angegeben, wird die Anzahl aller Artikel für jede Kategorie vereinheitlicht. Dies bedeutet, dass die niedrigste Anzahl an Artikeln einer Kategorie als Basiswert genommen wird und alle anderen Kategorien solange Artikel entfernen, bis sie die gleiche Anzahl an Artikeln haben. Damit kann ein ausgeglichenes Korpus erstellt werden.<br>

`--tokenization`: Wird dieses Argument angegeben, werden die Artikeltexte tokenisiert. Dies geschieht mit der NLTK-Funktion `word_tokenize`.<br>

`--german_translation`: Wird dieses Argument angegeben, werden deutsche Übersetzung in Klammern wie "(deutsch: ...)" erkannt und anstelle des fremdsprachigen Ausdrucks vor den Klammern verwendet. Dies wird durch das folgende Beispiel deutlich:<br>

<u>Beispiel</u>:<br>
„An mehreren Stellen im Film ist der Satz „Who watches the Watchmen?“ (deutsch: „Wer überwacht die Wächter?“) zu sehen, auf den sich auch der Titel des Comicromans bezieht.“<br>
→  „An mehreren Stellen im Film ist der Satz „Wer überwacht die Wächter?“ zu sehen, auf den sich auch der Titel des Comicromans bezieht.“<br>

`--nonlatin`: Wird dieses Argument angegeben, werden nicht-lateinische Zeichen entfernt.<br>

`--no_umlauts`: Wird dieses Argument angegeben, werden Umlaute ersetzt, etwa "ä" durch "ae".<br>

`--save_methods`: Wird dieses Argument angegeben, werden die Abkürzungen der Nachbearbeitungsmaßnahmen an den Dateiennamen gehängt.